# A New Performance-Complexity Landscape
##  JIT-based Fusion for Matrix-Free Multigrid Solvers, with a Case Study in Bonded Granular Media

## **Jed Brown**, Valeria Barra, Natalie N. Beams, Yohann Dudouit, Leila Ghaffari, Rezgar Shakeri, Ren Stengel, Jeremy L. Thompson


## SIAM PP22: MS13 Versatile Solvers on GPU-Based HPC Architectures

## Abstract

Abstract. Matrix-free multigrid solvers are an increasingly necessary to efficiently use modern hardware for science and engineering. Just-in-Time (JIT) compilation with kernel fusion makes it possible to separate concerns while making debuggable high-performance code, but this fusion may only be beneficial when the fused kernels can be scheduled with acceptable occupancy. We'll investigate abstraction leaks in convergence and performance tuning of matrix-free multigrid solvers as the problem specification and complexity of material models changes, all via a case study in bonded granular media.

In [2]:
from IPython.display import SVG, Video, HTML, IFrame
import pandas as pd
import altair as alt
from io import StringIO
import numpy as np


<img src="figures/app-perf-cartoon-2.png" width="55%" />

# Myths about high order methods

## 🛑 High order is more expensive per degree of freedom.
## 🛑 High order is no more accurate than low order if your problem has singularities.
## 🛑 High order doesn't help if you only need engineering tolerances.

# Constants matter!

# Why matrix-free?

* Assembled matrices need at least 4 bytes transferred per flop. Hardware does 10 flops per byte.
* Matrix-free methods store and move less data, compute faster.

<img src="figures/karlrupp/flop-per-byte-dp-2022.svg" class="floatleft" />
<img src="figures/assemble-or-not.svg" class="floatright" />

# Matrix-free methods are crucial for high order

In [3]:
df = pd.DataFrame([ # data from ceed/ceed/papers/ecp-special-gpu/figures/libceed/ceedpavsfa.tex
["CuSparse", 1, 1755570000],
["CuSparse", 2, 1230490000],
["CuSparse", 3, 720458000],
["CuSparse", 4, 248439000],
["CuSparse", 5, 172816000],
["CuSparse", 6, 103112000],
["CuSparse", 7, 83081600],
["CuSparse", 8, 40645000],
["libCEED", 1, 1572540000],
["libCEED", 2, 3616390000],
["libCEED", 3, 3937560000],
["libCEED", 4, 4134010000],
["libCEED", 5, 4062560000],
["libCEED", 6, 4148760000],
["libCEED", 7, 3579790000],
["libCEED", 8, 3839100000],
], columns=["code", "polynomial degree", "DoF/s"])
base = alt.Chart(df).encode(
    alt.X("polynomial degree:O"),
    alt.Y("DoF/s"),
)
alt.layer(
    base.mark_point().encode(shape="code", color="code"),
    base.mark_line().encode(alt.Color("code", legend=None)),
).resolve_scale(
    color="independent",
    shape="independent",
).properties(width=1200, height=800)

alt.LayerChart(...)

## [libCEED](https://libceed.readthedocs.io): fast algebra for finite elements

* Backend plugins with run-time selection
  * debug/memcheck, optimized
  * libxsmm, CUDA, HIP
  * MAGMA to CUDA and HIP
  * OCCA to OpenMP, OpenCL, CUDA, and HIP
* Single source vanilla C for QFunctions
  * Easy to debug, understand locally, C++ optional
  * Target for DSLs, AD
* Python, Julia, Rust
* 2-clause BSD
* Available via MFEM, PETSc, Nek5000

<img src="figures/ceed/libceed-backends-tex.png" />

Thanks to many developers, including Jeremy Thompson, Yohann Dudouit, Valeria Barra, Natalie Beams,  Ahmad Abdelfattah, Leila Ghaffari, Will Pazner, Thilina Ratnayaka, Tzanio Kolev, Veselin Dobrev, David Medina

<img src="figures/ceed/libCEED-2.png" width=100% />


## Quadrature functions: the math

\begin{gather*}
    v^T F(u) \sim \int_\Omega v \cdot \color{olive}{f_0(u, \nabla u)} + \nabla v \!:\! \color{olive}{f_1(u, \nabla u)} \quad
    v^T J w \sim \int_\Omega \begin{bmatrix} v \\ \nabla v \end{bmatrix}^T \color{teal}{\begin{bmatrix} f_{0,0} & f_{0,1} \\ f_{1,0} & f_{1,1} \end{bmatrix}}
    \begin{bmatrix} w \\ \nabla w \end{bmatrix} \\
    u = B_I \mathcal E_e u_L \qquad \nabla u = \frac{\partial X}{\partial x} B_{\nabla} \mathcal E_e u_L \\
    J w = \sum_e \mathcal E_e^T \begin{bmatrix} B_I \\ B_{\nabla} \end{bmatrix}^T
    \underbrace{\begin{bmatrix} I & \\ & \left( \frac{\partial X}{\partial x}\right)^T \end{bmatrix} W_q \color{teal}{\begin{bmatrix} f_{0,0} & f_{0,1} \\ f_{1,0} & f_{1,1} \end{bmatrix}} \begin{bmatrix} I & \\ & \left( \frac{\partial X}{\partial x}\right) \end{bmatrix}}_{\text{coefficients at quadrature points}} \begin{bmatrix} B_I \\ B_{\nabla} \end{bmatrix} \mathcal E_e w_L
\end{gather*}
  
* $B_I$ and $B_\nabla$ are tensor contractions -- independent of element geometry
* Choice of how to order and represent gathers $\mathcal E$ and scatters $\mathcal E^T$
* Similar for Neumann/Robin and nonlinear boundary conditions

## Quadrature functions: debuggable, vectorizable, and JITable

* Independent operations at each of `Q` quadrature points, order unspecified

```c
int L2residual(void *ctx, CeedInt Q,
    const CeedScalar *const in[],
    CeedScalar *const out[]) {
  const CeedScalar *u = in[0], *rho = in[1], *target = in[2];
  CeedScalar *v = out[0];
  for (CeedInt i=0; i<Q; i++)
    v[i] = rho[i] * (u[i] - target[i]);
  return 0;
}
```

![](figures/ceed/solids-perf-disassembly.png)

# Bonded particulate experiments (UT Dallas, Mines)

<img src="figures/micromorph/utdallas-glass-beads.png" />

<img src="figures/micromorph/logo-footer.png" class="floatleft66" />

# Verification and efficiency testing: Linear MMS

<img src="figures/ceed/solids-mms-conv.svg" width="80%" />
<img src="figures/ceed/solids-sing-conv.png" width="70%" />

![](figures/ceed/solids-eccomas/error-cost.svg)

# [Hyperelastic solids](https://libceed.readthedocs.io/en/latest/examples/solids/)

* $p$-multigrid, low-memory repr of matrix-free Jacobian
* Multi-node GPU on CUDA and ROCm

<img src="figures/ceed/libceed-solids-twist.gif" class="floatleft" />
<img src="figures/micromorph/libceed-epoxy-traction-20210829.gif" class="floatright" />

# How much resolution of what kind for what error?

In [42]:
Video("figures/ratel/one-hole-traction-20220222.ogv", html_attributes="autoplay")

<img src="figures/ratel/p-refinement.svg" width="100%" />

# Scaling with practical resolution

In [43]:
Video("figures/ratel/holes-20x20-q1-r0-20220222.ogv", html_attributes="autoplay loop")

## What's the weakest link?

| Preconditioned operator | Condition number |
|---|---|
| $p=1$: Just the coarse operator with GAMG | 17 |
| $p=2$: $p$-multigrid with GAMG solve | 17 |
| $p=2$: $p$-multigrid with Cholesky coarse | 1.6 |
| $p=3$: $p$-multigrid with GAMG coarse | 22 |
| $p=3$: $p$-multigrid with Cholesky coarse | 2.1 |

## Profiling on Crusher

* I was too slow to get performance numbers cleared; will share soon.

# BP performance on CPU (2x EPYC 7452)

In [16]:
from postprocess_base import read_logs
import altair as alt
from glob import glob

runs = read_logs(glob('data/ceed/**/*.txt'))
runs['FE_nodes_per_compute_node'] = runs['num_unknowns'] / (runs['num_procs'] / runs['num_procs_node']) / runs['dof_per_node']
runs.head()

,file,backend,backend_memtype,hostname,test,num_procs,num_procs_node,degree,quadrature_pts,code,bp,case,num_unknowns,num_elem,dof_per_node,ksp_its,time_per_it,cg_iteration_dps,FE_nodes_per_compute_node
0,data/ceed/lassen/lassen-16-4.txt,/gpu/cuda/gen,device,lassen410,PETSc CEED Benchmark Problem 1,1,1,1,3,libCEED,1,scalar,5616,4692,1,5,0.000362,15509500.0,5616.0
1,data/ceed/lassen/lassen-16-4.txt,/gpu/cuda/gen,device,lassen410,PETSc CEED Benchmark Problem 2,1,1,1,3,libCEED,2,vector,16848,4692,3,5,0.000366,46092000.0,5616.0
2,data/ceed/lassen/lassen-16-4.txt,/gpu/cuda/gen,device,lassen410,PETSc CEED Benchmark Problem 3,1,1,1,3,libCEED,3,scalar,3872,4692,1,1,0.000499,7762750.0,3872.0
3,data/ceed/lassen/lassen-16-4.txt,/gpu/cuda/gen,device,lassen410,PETSc CEED Benchmark Problem 4,1,1,1,3,libCEED,4,vector,11616,4692,3,1,0.000517,22485700.0,3872.0
4,data/ceed/lassen/lassen-16-4.txt,/gpu/cuda/gen,device,lassen410,PETSc CEED Benchmark Problem 1,1,1,1,3,libCEED,1,scalar,10800,9384,1,5,0.000363,29759700.0,10800.0


In [17]:
highlight = alt.selection_single(
    on='mouseover',
    fields=['degree', 'time_per_it', 'backend', 'hostname'],
    nearest=True,
    empty='none',
)

bps_select = alt.selection_single(
    fields=['bp'],
)

base = alt.Chart(runs[runs.hostname == "noether"]).encode(
    alt.Y('mdofs:Q', title='MDoF/s per CG iteration'),
    alt.Color('degree:N'),
    alt.Size('num_unknowns', scale=alt.Scale(type='log', domain=(1e3, 1e6))),
    alt.Shape('bp:N'),
    tooltip=('hostname', 'bp', 'num_procs', 'backend', 'num_elem', 'degree', 'num_unknowns', 'file'),
).transform_filter(
    bps_select,
).transform_calculate(
    mdofs='datum.cg_iteration_dps/1e6',
)

points = base.mark_point().encode(
    opacity=alt.condition(highlight, alt.value(1), alt.value(.5)),
).add_selection(
    highlight,
)

lines = base.mark_line().encode(
    size=alt.condition(alt.datum.degree - highlight.degree == 0, alt.value(2), alt.value(1))
)

pane = points + lines

composite = (
    pane.encode(
        alt.X('time_per_it', scale=alt.Scale(type='log'), title='Time per Iteration'),
    ).properties(width=600, height=600) |
    pane.encode(
        alt.X('FE_nodes_per_compute_node', scale=alt.Scale(type='log', domain=(3e4, 1e7), clamp=True), title='FE Nodes per Compute Node'),
    ).properties(width=600, height=600)
)

activator = alt.Chart(runs).mark_point().encode(
    alt.Y('bp', title='BP'),
    alt.Shape('bp')
).add_selection(bps_select).properties(title='Selection')

activator | composite.properties(title='CEED BPs')

alt.HConcatChart(...)

# BP performance on GPU (V100)

In [18]:
base = alt.Chart(runs[runs.hostname == "lassen385"]).encode(
    alt.Y('mdofs:Q', title='MDoF/s per CG iteration'),
    alt.Color('degree:N'),
    alt.Size('num_unknowns', scale=alt.Scale(type='log', domain=(1e3, 1e6))),
    alt.Shape('bp:N'),
    tooltip=('hostname', 'bp', 'num_procs', 'backend', 'num_elem', 'degree', 'num_unknowns', 'file'),
).transform_filter(
    bps_select,
).transform_calculate(
    mdofs='datum.cg_iteration_dps/1e6',
)

points = base.mark_point().encode(
    opacity=alt.condition(highlight, alt.value(1), alt.value(.5)),
).add_selection(
    highlight,
)

lines = base.mark_line().encode(
    size=alt.condition(alt.datum.degree - highlight.degree == 0, alt.value(2), alt.value(1))
)

pane = points + lines

composite = (
    pane.encode(
        alt.X('time_per_it', scale=alt.Scale(type='log'), title='Time per Iteration'),
    ).properties(width=600, height=600) |
    pane.encode(
        alt.X('FE_nodes_per_compute_node', scale=alt.Scale(type='log', domain=(3e4, 1e7), clamp=True), title='FE Nodes per Compute Node'),
    ).properties(width=600, height=600)
)

activator = alt.Chart(runs).mark_point().encode(
    alt.Y('bp', title='BP'),
    alt.Shape('bp')
).add_selection(bps_select).properties(title='Selection')

activator | composite.properties(title='CEED BPs')

alt.HConcatChart(...)

# libCEED and PETSc asynchrony

![](figures/ceed/libceed-bp3-nsys-cuda-gen.png)

# Anatomy of a matrix-light $p$-multigrid solve ($p=2$)

In [41]:
IFrame("figures/ratel/holes-q1-r0-flame.svg", width="1800", height="900")

# Anatomy from the inside looking out

In [21]:
IFrame("figures/ratel/holes-q1-r0-icicle.svg", width="1600", height="900")

# Solids: efficient matrix-free Jacobians, cf. [Davydov et al. (2020)](https://doi.org/10.1002/nme.6336)

<img src="figures/ceed/libceed-solids-initial-current.png" width="80%" />
<img src="figures/ceed/libceed-solids-jacobian-table.png" width="80%" />

<img src="figures/phypid/jeremy-thompson.jpeg" class="floatright10" />

# Preconditioners and local Fourier analysis

<img src="figures/lfatoolkit/bddc-cartoon.png" />

<img src="figures/lfatoolkit/lowVsHighDirichletBounds.png" width="80%" />

$$\kappa \le C \Big(1 + \log\big(p^2 \frac H h\big)\Big)^2$$

* https://github.com/jeremylt/LFAToolkit.jl
* Prior work in SISC 2019, SISC 2021

# Outlook

* Mesh geometry as coarsely as possible using quadratic (or higher) elements as necessary
    * Choose solution order to reach accuracy tolerance
    * When in doubt, make the Pareto diagram
* Write physics with any necessary storage at quadrature points
    * Same code provides matrix-free Jacobian and coarse matrix assembly
    * Perf model: bandwidth limited
    * Enzyme for automatic Jacobian action
    * Gotchas: `log1p` prevents vectorization on host (we replaced with series expansion)
* $p$-multigrid with assembly of linear operator for AMG
    * Robust matrix-free smoothers
    * Strong AMG (it's a small problem, but needs to be accurate)

Thanks to DOE ASCR, DOE ECP, and DOE PSAAP III for support.